In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("white")
%matplotlib inline

In [2]:
donations_cohort_with_cycle=pd.read_pickle('donations_cohort_with_cycle.pkl')

In [3]:
donations_cohort_with_cycle.groupby('cohort_gift_pattern').sum()

,lapsed_count,recovered_count,max_consec_giving_years,total_giving_years,count_2013,count_2014,count_2015,count_2016,count_2017,count_yr0,...,amount_05_2017,amount_06_2017,amount_07_2017,amount_08_2017,amount_09_2017,amount_10_2017,amount_11_2017,amount_12_2017,amount_total,count_total
cohort_gift_pattern,,,,,,,,,,,,,,,,,,,,,
"0,",3246751,1698013,1696680,1698013,281367,299265,343664,459121,568128,2160468,...,1513719.92,1346215.27,2430744.12,4660638.35,3614630.25,3381341.86,2653097.34,2732646.05,1.108747e+08,1951545
"0,1,",456484,324471,324008,324471,64778,102879,90901,133044,189491,351927,...,669986.44,470229.87,681269.53,1379156.40,1285395.33,1154510.77,1301913.91,1650338.26,3.663071e+07,581093
"0,1,2,",140611,113307,112917,113307,26461,47280,63866,83705,113547,111393,...,406391.15,286160.91,440258.05,652367.16,609036.92,504306.19,595653.51,820301.07,2.145994e+07,334859
"0,1,2,3,",64826,55642,55124,55642,18753,42483,56392,62173,59272,56260,...,256549.75,169966.88,258912.92,507154.02,413960.30,429247.78,547307.06,651570.56,1.679730e+07,239073
"0,1,2,3,4,",75395,70626,70626,70626,106916,174536,150126,143233,153440,127899,...,540893.94,429600.39,628210.06,1041223.65,976890.30,977064.73,1138716.31,1550895.28,4.428904e+07,728251
"0,1,2,4,",17175,12431,8901,12431,10974,12197,8673,1470,6534,12653,...,26429.85,17497.27,20652.61,47481.46,51619.98,39111.52,63445.55,109491.95,2.361383e+06,39848
"0,1,3,",39132,24601,16268,24601,6848,13560,10706,8209,6029,17694,...,16821.11,15506.57,26961.28,52175.80,48118.31,30767.99,32472.96,69326.56,2.995362e+06,45352
"0,1,3,4,",16037,11830,6200,11830,5221,8010,2292,5243,10504,7792,...,36626.51,28702.58,34569.66,57856.73,68852.78,51097.72,65494.68,135615.08,2.091935e+06,31270
"0,1,4,",14173,9353,5988,9353,6417,6798,1454,1,4562,8262,...,15205.59,9166.41,13938.24,34097.56,29615.97,22492.52,34206.41,44463.84,1.064790e+06,19232


In [4]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(donations_cohort_with_cycle.iloc[41])

id                                     0001107b9faa5c3bb42cfcecece1d587
first_gift_year                                              2013-01-01
lapsed_count                                                          6
recovered_count                                                       6
max_consec_giving_years                                               6
total_giving_years                                                    6
count_2013                                                            7
count_2014                                                           10
count_2015                                                           12
count_2016                                                            8
count_2017                                                            6
count_yr0                                                             7
count_yr1                                                            10
count_yr2                                                       

In [5]:
for col in donations_cohort_with_cycle.columns:
    print(col)

id
first_gift_year
lapsed_count
recovered_count
max_consec_giving_years
total_giving_years
count_2013
count_2014
count_2015
count_2016
count_2017
count_yr0
count_yr1
count_yr2
count_yr3
count_yr4
initial
first_gift
second_year
second_with_gap
multi_year
key_multi_year
recovered_multi_year
inconsistent_multi_year 
recovered_key_multi_year
lapsed_after_first
lapsed_after_first_long
lapsed_after_second
lapsed_multi_year
lapsed_long_multi_year
lapsed_key_multi_year
lapsed_long
cohort_0_first_gift
cohort_1_lapsed_after_first
cohort_1_second_year
cohort_2_lapsed_after_first_long
cohort_2_lapsed_after_second
cohort_2_multi_year
cohort_2_second_with_gap
cohort_3_first_gift
cohort_3_inconsistent_multi_year
cohort_3_key_multi_year
cohort_3_lapsed_after_second
cohort_3_lapsed_long
cohort_3_lapsed_multi_year
cohort_4_first_gift
cohort_4_inconsistent_multi_year
cohort_4_key_multi_year
cohort_4_lapsed_after_first
cohort_4_lapsed_key_multi_year
cohort_4_lapsed_long
cohort_4_lapsed_long_multi_year
coh

In [43]:
donations_cohort_with_cycle.groupby('first_gift_year').sum()['amount_yr1'].round(2)

first_gift_year
2012-01-01       16421.14
2013-01-01    15267703.98
2014-01-01     7745897.26
2015-01-01     8280549.87
2016-01-01    10475760.27
2017-01-01     4549913.20
2018-01-01           0.00
Name: amount_yr1, dtype: float64

In [7]:
donations_cohort_with_cycle.query('cohort_3_first_gift > 0').cohort_gift_pattern.value_counts()

0,3,     17619
0,3,4     3108
Name: cohort_gift_pattern, dtype: int64

In [8]:
print(f'cohort_1_lapsed_after_first: ', donations_cohort_with_cycle.query('cohort_1_lapsed_after_first > 0').cohort_gift_pattern.value_counts())
print(f'cohort_1_second_year: ', donations_cohort_with_cycle.query('cohort_1_second_year > 0').cohort_gift_pattern.value_counts())


cohort_1_lapsed_after_first:  0,          1548735
0,2,          44403
0,3,          17619
0,2,3,         7626
0,4,           6912
0,3,4          3108
0,2,3,4,       2735
0,2,4,         2035
Name: cohort_gift_pattern, dtype: int64
cohort_1_second_year:  0,1,          162004
0,1,2,         37639
0,1,2,3,       13781
0,1,2,3,4,     13102
0,1,3,          8134
0,1,4,          2994
0,1,2,4,        2967
0,1,3,4,        2815
Name: cohort_gift_pattern, dtype: int64


In [9]:

print(f'cohort_2_lapsed_after_first_long: ', donations_cohort_with_cycle.query('cohort_2_lapsed_after_first_long > 0').cohort_gift_pattern.value_counts())
print(f'cohort_2_lapsed_after_second: ', donations_cohort_with_cycle.query('cohort_2_lapsed_after_second > 0').cohort_gift_pattern.value_counts())
print(f'cohort_2_multi_year: ', donations_cohort_with_cycle.query('cohort_2_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_2_second_with_gap: ', donations_cohort_with_cycle.query('cohort_2_second_with_gap > 0').cohort_gift_pattern.value_counts())


cohort_2_lapsed_after_first_long:  0,       1143733
0,3,       17619
0,4,        6912
0,3,4       3108
Name: cohort_gift_pattern, dtype: int64
cohort_2_lapsed_after_second:  0,1,        132013
0,1,3,        8134
0,1,4,        2994
0,1,3,4,      2815
Name: cohort_gift_pattern, dtype: int64
cohort_2_multi_year:  0,1,2,        37639
0,1,2,3,      13781
0,1,2,3,4,    13102
0,1,2,4,       2967
Name: cohort_gift_pattern, dtype: int64
cohort_2_second_with_gap:  0,2,        44403
0,2,3,       7626
0,2,3,4,     2735
0,2,4,       2035
Name: cohort_gift_pattern, dtype: int64


In [10]:
print(f'cohort_3_first_gift: ', donations_cohort_with_cycle.query('cohort_3_first_gift > 0').cohort_gift_pattern.value_counts())
print(f'cohort_3_inconsistent_multi_year: ', donations_cohort_with_cycle.query('cohort_3_inconsistent_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_3_key_multi_year: ', donations_cohort_with_cycle.query('cohort_3_key_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_4_lapsed_after_second: ', donations_cohort_with_cycle.query('cohort_3_lapsed_after_second > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_lapsed_long: ', donations_cohort_with_cycle.query('cohort_3_lapsed_long > 0').cohort_gift_pattern.value_counts())

print(f'cohort_4_lapsed_multi_year: ', donations_cohort_with_cycle.query('cohort_3_lapsed_multi_year > 0').cohort_gift_pattern.value_counts())


cohort_3_first_gift:  0,3,     17619
0,3,4     3108
Name: cohort_gift_pattern, dtype: int64
cohort_3_inconsistent_multi_year:  0,1,3,      8134
0,2,3,      7626
0,1,3,4,    2815
0,2,3,4,    2735
Name: cohort_gift_pattern, dtype: int64
cohort_3_key_multi_year:  0,1,2,3,      13781
0,1,2,3,4,    13102
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_after_second:  0,2,      36065
0,2,4,     2035
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_long:  0,        763012
0,1,       85811
0,4,        6912
0,1,4,      2994
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_multi_year:  0,1,2,      27304
0,1,2,4,     2967
Name: cohort_gift_pattern, dtype: int64


In [11]:
print(f'cohort_4_first_gift: ', donations_cohort_with_cycle.query('cohort_4_first_gift > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_inconsistent_multi_year: ', donations_cohort_with_cycle.query('cohort_4_inconsistent_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_key_multi_year: ', donations_cohort_with_cycle.query('cohort_4_key_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_4_lapsed_after_first: ', donations_cohort_with_cycle.query('cohort_4_lapsed_after_first > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_lapsed_key_multi_year: ', donations_cohort_with_cycle.query('cohort_4_lapsed_key_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_lapsed_long: ', donations_cohort_with_cycle.query('cohort_4_lapsed_long > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_lapsed_long_multi_year: ', donations_cohort_with_cycle.query('cohort_4_lapsed_long_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_4_lapsed_multi_year: ', donations_cohort_with_cycle.query('cohort_4_lapsed_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_multi_year: ', donations_cohort_with_cycle.query('cohort_4_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_recovered_multi_year: ', donations_cohort_with_cycle.query('cohort_4_recovered_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_4_second_year: ', donations_cohort_with_cycle.query('cohort_4_second_year > 0').cohort_gift_pattern.value_counts())


cohort_4_first_gift:  0,4,      6912
0,1,4,    2994
Name: cohort_gift_pattern, dtype: int64
cohort_4_inconsistent_multi_year:  0,2,4,    2035
Name: cohort_gift_pattern, dtype: int64
cohort_4_key_multi_year:  0,1,2,3,4,    13102
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_after_first:  0,3,    14084
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_key_multi_year:  0,1,2,3,    9184
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_long:  0,      474008
0,1,     56793
0,2,     21660
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_long_multi_year:  0,1,2,    16578
Name: cohort_gift_pattern, dtype: int64
cohort_4_lapsed_multi_year:  0,1,3,    6397
0,2,3,    5511
Name: cohort_gift_pattern, dtype: int64
cohort_4_multi_year:  0,1,3,4,    2815
0,2,3,4,    2735
Name: cohort_gift_pattern, dtype: int64
cohort_4_recovered_multi_year:  0,1,2,4,    2967
Name: cohort_gift_pattern, dtype: int64
cohort_4_second_year:  0,3,4    3108
Name: cohort_gift_pattern, dtype: int64

In [24]:
print(f'cohort_5_first_gift: ', donations_cohort_with_cycle.query('cohort_5_first_gift > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_inconsistent_multi_year: ', donations_cohort_with_cycle.query('cohort_5_inconsistent_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_key_multi_year: ', donations_cohort_with_cycle.query('cohort_5_key_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_5_lapsed_after_first: ', donations_cohort_with_cycle.query('cohort_5_lapsed_after_first > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_lapsed_after_first_long: ', donations_cohort_with_cycle.query('cohort_5_lapsed_after_first_long > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_lapsed_after_second: ', donations_cohort_with_cycle.query('cohort_5_lapsed_after_second > 0').cohort_gift_pattern.value_counts())

print(f'cohort_5_lapsed_key_multi_year: ', donations_cohort_with_cycle.query('cohort_5_lapsed_key_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_lapsed_long: ', donations_cohort_with_cycle.query('cohort_5_lapsed_long > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_lapsed_long_key_multi_year: ', donations_cohort_with_cycle.query('cohort_5_lapsed_long_key_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_lapsed_long_multi_year: ', donations_cohort_with_cycle.query('cohort_5_lapsed_long_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_5_lapsed_multi_year: ', donations_cohort_with_cycle.query('cohort_5_lapsed_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_multi_year: ', donations_cohort_with_cycle.query('cohort_5_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_recovered_key_multi_year: ', donations_cohort_with_cycle.query('cohort_5_recovered_key_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_recovered_multi_year: ', donations_cohort_with_cycle.query('cohort_5_recovered_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_5_second_with_gap: ', donations_cohort_with_cycle.query('cohort_5_second_with_gap > 0').cohort_gift_pattern.value_counts())
print(f'cohort_5_second_year: ', donations_cohort_with_cycle.query('cohort_5_second_year > 0').cohort_gift_pattern.value_counts())

cohort_5_first_gift:  0,        1332
0,1,       463
0,1,2,     390
0,2,       254
Name: cohort_gift_pattern, dtype: int64
cohort_5_inconsistent_multi_year:  0,1,2,4,    562
Name: cohort_gift_pattern, dtype: int64
cohort_5_key_multi_year:  0,1,2,3,4,    5115
0,2,3,4,       606
0,1,3,4,       570
Name: cohort_gift_pattern, dtype: int64
cohort_5_lapsed_after_first:  0,4,      4508
0,1,4,    1826
Name: cohort_gift_pattern, dtype: int64
cohort_5_lapsed_after_first_long:  0,3,    6462
Name: cohort_gift_pattern, dtype: int64
cohort_5_lapsed_after_second:  0,3,4    1740
Name: cohort_gift_pattern, dtype: int64
cohort_5_lapsed_key_multi_year:  0,1,2,3,4,    4767
Name: cohort_gift_pattern, dtype: int64
cohort_5_lapsed_long:  0,      226577
0,1,     28372
0,2,     10138
Name: cohort_gift_pattern, dtype: int64
cohort_5_lapsed_long_key_multi_year:  0,1,2,3,    3925
Name: cohort_gift_pattern, dtype: int64
cohort_5_lapsed_long_multi_year:  0,1,2,    9038
0,1,3,    2878
0,2,3,    2231
Name: cohort_gift

In [25]:
print(f'cohort_6_first_gift: ', donations_cohort_with_cycle.query('cohort_6_first_gift > 0').cohort_gift_pattern.value_counts())
print(f'cohort_6_key_multi_year: ', donations_cohort_with_cycle.query('cohort_6_key_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_6_lapsed_after_first: ', donations_cohort_with_cycle.query('cohort_6_lapsed_after_first > 0').cohort_gift_pattern.value_counts())
print(f'cohort_6_lapsed_after_first_long: ', donations_cohort_with_cycle.query('cohort_6_lapsed_after_first_long > 0').cohort_gift_pattern.value_counts())
print(f'cohort_6_lapsed_after_second: ', donations_cohort_with_cycle.query('cohort_6_lapsed_after_second > 0').cohort_gift_pattern.value_counts())

print(f'cohort_6_lapsed_key_multi_year: ', donations_cohort_with_cycle.query('cohort_6_lapsed_key_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_6_lapsed_long: ', donations_cohort_with_cycle.query('cohort_6_lapsed_long > 0').cohort_gift_pattern.value_counts())
print(f'cohort_6_lapsed_long_key_multi_year: ', donations_cohort_with_cycle.query('cohort_6_lapsed_long_key_multi_year > 0').cohort_gift_pattern.value_counts())
print(f'cohort_6_lapsed_long_multi_year: ', donations_cohort_with_cycle.query('cohort_6_lapsed_long_multi_year > 0').cohort_gift_pattern.value_counts())

print(f'cohort_6_multi_year: ', donations_cohort_with_cycle.query('cohort_6_multi_year > 0').cohort_gift_pattern.value_counts())

cohort_6_first_gift:  0,    1
Name: cohort_gift_pattern, dtype: int64
cohort_6_key_multi_year:  0,1,2,3,4,    1
Name: cohort_gift_pattern, dtype: int64
cohort_6_lapsed_after_first:  0,    3
Name: cohort_gift_pattern, dtype: int64
cohort_6_lapsed_after_first_long:  0,1,4,    1
Name: cohort_gift_pattern, dtype: int64
cohort_6_lapsed_after_second:  0,3,    1
Name: cohort_gift_pattern, dtype: int64
cohort_6_lapsed_key_multi_year:  0,1,2,3,4,    2
Name: cohort_gift_pattern, dtype: int64
cohort_6_lapsed_long:  0,      98
0,1,    18
0,3,     3
0,2,     2
Name: cohort_gift_pattern, dtype: int64
cohort_6_lapsed_long_key_multi_year:  0,1,2,3,4,    1
Name: cohort_gift_pattern, dtype: int64
cohort_6_lapsed_long_multi_year:  0,1,2,      5
0,1,3,      2
0,1,2,4,    1
0,1,2,3,    1
Name: cohort_gift_pattern, dtype: int64
cohort_6_multi_year:  0,1,2,4,    1
Name: cohort_gift_pattern, dtype: int64


In [30]:
donations_cohort_with_cycle.query('cohort_6_multi_year > 0')

,id,first_gift_year,lapsed_count,recovered_count,max_consec_giving_years,total_giving_years,count_2013,count_2014,count_2015,count_2016,...,amount_06_2017,amount_07_2017,amount_08_2017,amount_09_2017,amount_10_2017,amount_11_2017,amount_12_2017,amount_total,count_total,cohort_gift_pattern
545726,44f5e8002149cc536599f46eb1a40139,2012-01-01,7,6,3,6,1,1,0,2,...,0.0,0.0,116.28,131.8,0.0,0.0,0.0,1083.9,6,"0,1,2,4,"


In [23]:
donations_cohort_with_cycle.query('second_year > 0 & first_gift_year == "2017-01-01"').sum()['amount_yr1']

4549913.200000005

In [15]:
c13=donations_cohort_with_cycle.query('first_gift_year =="2013-01-01"')

In [16]:
c13.cohort_0_first_gift.sum()

320069

In [17]:
c13.cohort_1_second_year.sum()/c13.cohort_0_first_gift.sum()*100

19.411439408377568

In [46]:
df=donations_cohort_with_cycle[[
'id',
'first_gift_year',
'initial',
'first_gift',
'second_year',
'second_with_gap',
'multi_year',
'key_multi_year',
'recovered_multi_year',
'inconsistent_multi_year ',
'recovered_key_multi_year',
'lapsed_after_first',
'lapsed_after_first_long',
'lapsed_after_second',
'lapsed_multi_year',
'lapsed_long_multi_year',
'lapsed_key_multi_year',
'lapsed_long',
'cohort_0_first_gift',
'cohort_1_lapsed_after_first',
'cohort_1_second_year',
'cohort_2_lapsed_after_first_long',
'cohort_2_lapsed_after_second',
'cohort_2_multi_year',
'cohort_2_second_with_gap',
'cohort_3_first_gift',
'cohort_3_inconsistent_multi_year',
'cohort_3_key_multi_year',
'cohort_3_lapsed_after_second',
'cohort_3_lapsed_long',
'cohort_3_lapsed_multi_year',
'cohort_4_first_gift',
'cohort_4_inconsistent_multi_year',
'cohort_4_key_multi_year',
'cohort_4_lapsed_after_first',
'cohort_4_lapsed_key_multi_year',
'cohort_4_lapsed_long',
'cohort_4_lapsed_long_multi_year',
'cohort_4_lapsed_multi_year',
'cohort_4_multi_year',
'cohort_4_recovered_multi_year',
'cohort_4_second_year',
'cohort_5_first_gift',
'cohort_5_inconsistent_multi_year',
'cohort_5_key_multi_year',
'cohort_5_lapsed_after_first',
'cohort_5_lapsed_after_first_long',
'cohort_5_lapsed_after_second',
'cohort_5_lapsed_key_multi_year',
'cohort_5_lapsed_long',
'cohort_5_lapsed_long_key_multi_year',
'cohort_5_lapsed_long_multi_year',
'cohort_5_lapsed_multi_year',
'cohort_5_multi_year',
'cohort_5_recovered_key_multi_year',
'cohort_5_recovered_multi_year',
'cohort_5_second_with_gap',
'cohort_5_second_year',
'cohort_6_first_gift',
'cohort_6_key_multi_year',
'cohort_6_lapsed_after_first',
'cohort_6_lapsed_after_first_long',
'cohort_6_lapsed_after_second',
'cohort_6_lapsed_key_multi_year',
'cohort_6_lapsed_long',
'cohort_6_lapsed_long_key_multi_year',
'cohort_6_lapsed_long_multi_year',
'cohort_6_multi_year'
]].copy()

In [47]:
df.head()

,id,first_gift_year,initial,first_gift,second_year,second_with_gap,multi_year,key_multi_year,recovered_multi_year,inconsistent_multi_year,...,cohort_6_first_gift,cohort_6_key_multi_year,cohort_6_lapsed_after_first,cohort_6_lapsed_after_first_long,cohort_6_lapsed_after_second,cohort_6_lapsed_key_multi_year,cohort_6_lapsed_long,cohort_6_lapsed_long_key_multi_year,cohort_6_lapsed_long_multi_year,cohort_6_multi_year
0,00000ce845c00cbf0686c992fc369df4,2013-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,00002783bc5d108510f3f9666c8b1edd,2016-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,00002d44003ed46b066607c5455a999a,2016-01-01,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,00002eb25d60a09c318efbd0797bffb5,2015-01-01,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,0000300773fe015f870914b42528541b,2017-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


### Start here

In [48]:
grouped=df.groupby('first_gift_year').sum().reset_index()

In [49]:
grouped

,first_gift_year,initial,first_gift,second_year,second_with_gap,multi_year,key_multi_year,recovered_multi_year,inconsistent_multi_year,recovered_key_multi_year,...,cohort_6_first_gift,cohort_6_key_multi_year,cohort_6_lapsed_after_first,cohort_6_lapsed_after_first_long,cohort_6_lapsed_after_second,cohort_6_lapsed_key_multi_year,cohort_6_lapsed_long,cohort_6_lapsed_long_key_multi_year,cohort_6_lapsed_long_multi_year,cohort_6_multi_year
0,2012-01-01,0,1,0,0,1,1,0,0,0,...,1,1,3,1,1,2,121,1,9,1.0
1,2013-01-01,0,2436,873,314,736,6288,380,561,518,...,0,0,0,0,0,0,0,0,0,0.0
2,2014-01-01,0,2699,913,0,1690,3220,628,582,0,...,0,0,0,0,0,0,0,0,0,0.0
3,2015-01-01,0,3535,0,0,0,4597,0,3852,0,...,0,0,0,0,0,0,0,0,0,0.0
4,2016-01-01,0,0,1,8338,10335,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
5,2017-01-01,0,0,29990,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
6,2018-01-01,0,147945,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


In [66]:
# here is the sql query I ran 

# select
# first_gift_year,

#sum(lifecycle_cohort_0_first_gift) as first_gift_cnt,
first_gift_cnt=grouped.cohort_0_first_gift

# sum(lifecycle_cohort_1_second_year)   as second_year_cnt,
second_year_cnt=grouped.cohort_1_second_year

# sum(lifecycle_cohort_2_lapsed_after_second)   as lapsed_after_second_cnt,
lapsed_after_second_cnt=grouped.cohort_2_lapsed_after_second

# sum(lifecycle_cohort_2_second_with_gap)   as second_with_gap_cnt,
second_with_gap_cnt=grouped.cohort_2_second_with_gap

# sum(lifecycle_cohort_2_lapsed_after_first_long)   as lapsed_after_first_long_cnt
lapsed_after_first_long_cnt=grouped.cohort_2_lapsed_after_first_long

# sum(lifecycle_cohort_2_multi_year)   as multi_year_cnt,
multi_year_cnt=grouped.cohort_2_multi_year


# ((sum(lifecycle_cohort_1_second_year * 1.0) ) / 
#  (sum(lifecycle_cohort_0_first_gift  * 1.0))) * 100.0 as second_year,
second_year=second_year_cnt/first_gift_cnt*100

# ((sum(lifecycle_cohort_1_second_year * cohort_amount_in_year_in_1) ) / 
#  (sum(lifecycle_cohort_1_second_year  * 1.0))) / 100.0 as second_year_avg,
second_year_avg=second_year_avg=(donations_cohort_with_cycle.groupby('first_gift_year').sum()['amount_yr1'].reset_index()['amount_yr1'])/second_year_cnt

# ((sum(lifecycle_cohort_1_lapsed_after_first * 1.0) ) / 
#  (sum(lifecycle_cohort_0_first_gift  * 1.0))) * 100.0 as lapsed_after_first,
lapsed_after_first=grouped.cohort_1_lapsed_after_first/first_gift_cnt*100

# ((sum(lifecycle_cohort_2_multi_year * 1.0) ) / 
#  (sum(lifecycle_cohort_1_second_year  * 1.0))) * 100.0 as multi_year,
multi_year=grouped.cohort_2_multi_year/second_year_cnt*100


# ((sum(lifecycle_cohort_2_multi_year * cohort_amount_in_year_in_2) ) / 
#  (sum(lifecycle_cohort_2_multi_year  * 1.0))) / 100.0 as multi_year_avg,
multi_year_avg=(donations_cohort_with_cycle.query('cohort_2_multi_year > 0').groupby('first_gift_year').sum()['amount_yr2'].reset_index()['amount_yr2'])/multi_year_cnt


# ((sum(lifecycle_cohort_2_lapsed_after_second * 1.0) ) / 
#  (sum(lifecycle_cohort_1_second_year  * 1.0))) * 100.0 as lapsed_after_second,
lapsed_after_second=lapsed_after_second_cnt/second_year_cnt*100

# ((sum(lifecycle_cohort_2_second_with_gap * 1.0) ) / 
#  (sum(lifecycle_cohort_1_lapsed_after_first  * 1.0))) * 100.0 as second_with_gap,
second_with_gap=second_with_gap_cnt/grouped.cohort_1_lapsed_after_first*100


# ((sum(lifecycle_cohort_2_second_with_gap * cohort_amount_in_year_in_2) ) / 
#  (sum(lifecycle_cohort_2_second_with_gap  * 1.0))) / 100.0 as second_with_gap_avg,
second_with_gap_avg=(donations_cohort_with_cycle.query('cohort_2_second_with_gap > 0').groupby('first_gift_year').sum()['amount_yr2'].reset_index()['amount_yr2'])/second_with_gap_cnt

# ((sum(lifecycle_cohort_2_lapsed_after_first_long * 1.0) ) / 
#  (sum(lifecycle_cohort_1_lapsed_after_first  * 1.0))) * 100.0 as lapsed_after_first_long,
lapsed_after_first_long=lapsed_after_first_long_cnt/grouped.cohort_1_lapsed_after_first*100



# from donationslifecycles
# group by first_gift_year

# which gives the attached csv which should show some of the power we can get in sql


In [75]:
donations_cohort_with_cycle.groupby('first_gift_year').sum()['amount_yr1'].reset_index()['amount_yr1']

0       16421.14
1    15267703.98
2     7745897.26
3     8280549.87
4    10475760.27
5     4549913.20
6           0.00
Name: amount_yr1, dtype: float64

In [77]:
donations_cohort_with_cycle.query('cohort_1_lapsed_after_first > 0').groupby('first_gift_year').sum()['amount_yr0'].reset_index()['amount_yr0'].round(2)


0       21842.64
1    17392582.86
2    19384701.32
3    23316118.31
4    27839492.42
5    31672744.45
Name: amount_yr0, dtype: float64

In [78]:
donations_cohort_with_cycle.query('cohort_1_second_year > 0').groupby('first_gift_year').sum()['amount_yr0'].reset_index()['amount_yr0'].round(2)


0        3000.00
1    13019180.74
2     6789581.20
3     6830760.01
4     8690722.10
5     6301026.03
Name: amount_yr0, dtype: float64

In [67]:
data = {
    'cohort_year':grouped.first_gift_year,
    'first_gift_cnt':first_gift_cnt,
    'second_year':second_year,
'second_year_cnt':second_year_cnt,
'second_year_avg':second_year_avg,
'lapsed_after_first':lapsed_after_first,
'multi_year':multi_year,
'multi_year_cnt':multi_year_cnt,
'multi_year_avg':multi_year_avg,
'lapsed_after_second':lapsed_after_second,
'lapsed_after_second_cnt':lapsed_after_second_cnt,
'second_with_gap':second_with_gap,
'second_with_gap_cnt':second_with_gap_cnt,
'second_with_gap_avg':second_with_gap_avg,
'lapsed_after_first_long':lapsed_after_first_long,
'lapsed_after_first_long_cnt':lapsed_after_first_long_cnt
}




In [68]:
data

{'cohort_year': 0    2012-01-01
 1    2013-01-01
 2    2014-01-01
 3    2015-01-01
 4    2016-01-01
 5    2017-01-01
 6    2018-01-01
 Name: first_gift_year, dtype: object, 'first_gift_cnt': 0       141
 1    320069
 2    320673
 3    355137
 4    445598
 5    434991
 6    147945
 Name: cohort_0_first_gift, dtype: int64, 'second_year': 0    23.404255
 1    19.411439
 2    15.176519
 3    12.974711
 4    12.688118
 5     6.894396
 6     0.000000
 dtype: float64, 'second_year_cnt': 0       33
 1    62130
 2    48667
 3    46078
 4    56538
 5    29990
 6        0
 Name: cohort_1_second_year, dtype: int64, 'second_year_avg': 0    497.610303
 1    245.738033
 2    159.161182
 3    179.707233
 4    185.287068
 5    151.714345
 6           NaN
 dtype: float64, 'lapsed_after_first': 0    76.595745
 1    80.588561
 2    84.823481
 3    87.025289
 4    87.311882
 5    93.105604
 6     0.000000
 dtype: float64, 'multi_year': 0    36.363636
 1    41.976501
 2    32.340189
 3    33.254482
 4    18

In [69]:
df4 = pd.DataFrame(data) 

In [70]:
df4

,cohort_year,first_gift_cnt,second_year,second_year_cnt,second_year_avg,lapsed_after_first,multi_year,multi_year_cnt,multi_year_avg,lapsed_after_second,lapsed_after_second_cnt,second_with_gap,second_with_gap_cnt,second_with_gap_avg,lapsed_after_first_long,lapsed_after_first_long_cnt
0,2012-01-01,141,23.404255,33,497.610303,76.595745,36.363636,12,1597.260833,63.636364,21,1.851852,2,32.140000,98.148148,106
1,2013-01-01,320069,19.411439,62130,245.738033,80.588561,41.976501,26080,407.968347,58.023499,36050,6.262333,16153,120.565399,93.737667,241786
2,2014-01-01,320673,15.176519,48667,159.161182,84.823481,32.340189,15739,299.721817,67.659811,32928,5.803548,15786,108.981826,94.196452,256220
3,2015-01-01,355137,12.974711,46078,179.707233,87.025289,33.254482,15323,366.020322,66.745518,30755,5.345258,16520,111.114785,94.654742,292539
4,2016-01-01,445598,12.688118,56538,185.287068,87.311882,18.279741,10335,205.318955,81.718490,46202,2.143114,8338,91.576951,97.856629,380721
5,2017-01-01,434991,6.894396,29990,151.714345,93.105604,0.000000,0,NaN,0.000000,0,0.000000,0,NaN,0.000000,0
6,2018-01-01,147945,0.000000,0,NaN,0.000000,NaN,0,NaN,NaN,0,NaN,0,NaN,NaN,0


In [71]:
df4.to_csv('df4.csv')

In [ ]:
grouped.cohort_1_second_year

In [17]:
donations_cohort_with_cycle.groupby('first_gift_year').agg(
    {'first_gift', 'second_year': "sum",  # get the count of networks
    }
)
df.groupby('Category').agg({'Item':'size','shop1':['sum','mean','std'],'shop2':['sum','mean','std'],'shop3':['sum','mean','std']})


SyntaxError: invalid syntax (<ipython-input-17-e2a269811057>, line 5)

In [ ]:
def filter_func(x):
    return x['data2'].std() > 4

In [31]:
##START HERE FRIDAY

In [39]:
donations_cohort_with_cycle1=donations_cohort_with_cycle[['id','first_gift_year','amount_yr0','amount_yr1','amount_yr2','amount_yr3','amount_yr4','cohort_0_first_gift',
'cohort_1_lapsed_after_first',
'cohort_1_second_year',
'cohort_2_lapsed_after_first_long',
'cohort_2_lapsed_after_second',
'cohort_2_multi_year',
'cohort_2_second_with_gap',
'cohort_3_first_gift',
'cohort_3_inconsistent_multi_year',
'cohort_3_key_multi_year',
'cohort_3_lapsed_after_second',
'cohort_3_lapsed_long',
'cohort_3_lapsed_multi_year',
'cohort_4_first_gift',
'cohort_4_inconsistent_multi_year',
'cohort_4_key_multi_year',
'cohort_4_lapsed_after_first',
'cohort_4_lapsed_key_multi_year',
'cohort_4_lapsed_long',
'cohort_4_lapsed_long_multi_year',
'cohort_4_lapsed_multi_year',
'cohort_4_multi_year',
'cohort_4_recovered_multi_year',
'cohort_4_second_year',
'cohort_gift_pattern'                                                          
]].copy()

In [40]:
df01234=donations_cohort_with_cycle1.query('cohort_gift_pattern == "0,1,2,3,4,"')

In [41]:
df01234

,id,first_gift_year,amount_yr0,amount_yr1,amount_yr2,amount_yr3,amount_yr4,cohort_0_first_gift,cohort_1_lapsed_after_first,cohort_1_second_year,...,cohort_4_key_multi_year,cohort_4_lapsed_after_first,cohort_4_lapsed_key_multi_year,cohort_4_lapsed_long,cohort_4_lapsed_long_multi_year,cohort_4_lapsed_multi_year,cohort_4_multi_year,cohort_4_recovered_multi_year,cohort_4_second_year,cohort_gift_pattern
41,0001107b9faa5c3bb42cfcecece1d587,2013-01-01,125.00,175.0,240.0,195.00,160.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
107,0002bcb9f69cd91cbeae163fc9cd874f,2013-01-01,150.00,75.0,58.0,236.00,1139.59,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
320,0009b8318c1914701e4d1063f2a008d8,2013-01-01,357.00,33.0,35.0,109.00,139.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
374,000baeedb9c9756b6335a16918770e0f,2013-01-01,20.00,50.0,50.0,50.00,75.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
688,0015dc157335e5ad6f569e5264bbf33e,2013-01-01,3756.36,189.5,110.0,50.00,300.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023991,ffec50d779286cac79d867a8eb5de741,2013-01-01,100.00,150.0,10.0,150.00,100.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
2023996,ffecc7551941f8850b616b7dc2e5c8fe,2014-01-01,100.00,50.0,300.0,50.00,50.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
2024029,ffeda789400d5addb91599d79da83ea8,2013-01-01,110.00,140.0,130.0,160.79,110.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"
2024118,fff0c19388fb85f152dce64748f7d86b,2014-01-01,10.00,20.0,60.0,200.00,50.00,1,0,1,...,1,0,0,0,0,0,0,0,0,"0,1,2,3,4,"


In [42]:
df01234.sum()

id                                  0001107b9faa5c3bb42cfcecece1d5870002bcb9f69cd9...
first_gift_year                     2013-01-012013-01-012013-01-012013-01-012013-0...
amount_yr0                                                                6.22259e+06
amount_yr1                                                                9.30266e+06
amount_yr2                                                                1.01037e+07
amount_yr3                                                                1.04825e+07
amount_yr4                                                                9.28515e+06
cohort_0_first_gift                                                             13102
cohort_1_lapsed_after_first                                                         0
cohort_1_second_year                                                            13102
cohort_2_lapsed_after_first_long                                                    0
cohort_2_lapsed_after_second                          